In [ ]:
from types import SimpleNamespace
import numpy as np
import deepdish as dd
import os
import holoviews as hv
import xarray as xr
import pandas as pd
hv.notebook_extension()
%opts Scatter (color='b') 
d = SimpleNamespace(phi=hv.Dimension(r'$\phi$', unit='rad'),
                    phi_c=hv.Dimension(r'$\phi_c$', unit='rad'),
                    I=hv.Dimension('$I$', unit=r'meV / $\Phi_0$'),
                    I_c=hv.Dimension('$I_c$', unit=r'meV / $\Phi_0$'),
                    B=hv.Dimension('$B$', unit='T'),
                    T=hv.Dimension('$T$', unit='K'), 
                    L=hv.Dimension('$L$', unit='nm'), 
                    g=hv.Dimension('$g$-factor'), 
                    alpha=hv.Dimension(r'$\alpha$', unit='mev nm'), 
                    mu=hv.Dimension(r'$\mu$', unit='meV'), 
                    a=hv.Dimension('$a$', unit='nm'),
                    V=hv.Dimension('$V$', unit='meV'))

pd.set_option('display.max_columns', None)

# Gate

In [ ]:
from itertools import product
df = pd.DataFrame()

files = [f for f in os.listdir('./') if f.startswith('current_as_function_of_gate_and_B_x__ga') and f.endswith('.h5')]
for fname in files:
    x = dd.io.load(fname)
    fname_new = fname[:-3] + '.hdf'
    vals = list(product(x.pop('B_x'), x.pop('Vs')))
    df1 = pd.DataFrame(vals, columns=['B_x', 'Vs'])
    df2 = pd.DataFrame(x.pop('current_phase'))
    df_new = pd.concat([df1, df2], axis=1)
    
    for dim in x.keys():
        if isinstance(x[dim], dict):
            dic = {key: val for key, val in x[dim].items() if val is not None}
            df_new = df_new.assign(**dic)
        else:
            df_new[dim] = x[dim]

    df = df.append(df_new, ignore_index=True)

# df = df.sort_values(by=['gate_size']) # doesn't work!
df.reindex().to_hdf('all_gate_data.hdf', 'all_data', mode='w')

In [ ]:
%%opts Image [colorbar=True aspect='square']

inds = ['gate_size', 'T', 'L', 'with_leads', 'orbital', 'g', 'alpha', 'mu', 'a']
kdims = ['gate_size', d.T, d.L, 'infinite leads', 'orbital', d.g, d.alpha, d.mu, d.a]

gb = df.groupby(inds)
bnds = (df['B_x'].min(), df['Vs'].min(), df['B_x'].max(), df['Vs'].max())
gate_sizes = hv.HoloMap({key: hv.Image(np.rot90(val['current_c'].reshape(100, -1)),
                                       bounds=bnds, kdims=[d.B, d.V], vdims=[d.I_c])
                         for key, val in gb}, kdims=kdims)


gate_sizes